In [1]:
UPDATE = "asynchronous"
DIRECTORY = "models/random_nk3"
NUM_RUNS = 100
DEBUG = True
OUTPUT_FILE = "basic_analysis.csv"

In [2]:
import os
import csv

import numpy as np
from pyboolnet.external.bnet2primes import bnet_text2primes
from pyboolnet.prime_implicants import percolate
from pyboolnet.file_exchange import primes2bnet
from pyboolnet.trap_spaces import compute_trap_spaces
from pyboolnet.state_transition_graphs import primes2stg

from grouping import sd_grouping, null_grouping, random_grouping
from scc_dags import get_scc_dag, get_attractor_states
from transition_matrix import get_transition_matrix
from matrix_operations import compress_matrix
from graph import get_markov_chain


In [3]:
def get_general_info(bnet, data={}, num_runs=100, update="asynchronous", DEBUG=False):

    # initialize data
    data["general"] = {}
    
    
    ### general information

    # read bnet and get primes, and sort them
    primes = bnet_text2primes(bnet)
    primes = {key: primes[key] for key in sorted(primes)}

    # get number of nodes
    data["general"]["N"] = len(primes)

    # get number of sources
    sources = 0
    for node in primes:
        if primes[node] == [[{node: 0}], [{node: 1}]]:
            sources += 1
    data["general"]["sources"] = sources

    # Percolate constant nodes and remove them
    percolated_primes = percolate(primes, remove_constants=True, copy=True)

    # get number of percolated nodes
    data["general"]["N_perc"] = len(percolated_primes)

    if len(percolated_primes) == 0:
        return data
    
    # get percolated bnet
    percolated_bnet = primes2bnet(percolated_primes)

    # get number of percolated sources
    sources_perc = 0
    for node in percolated_primes:
        if percolated_primes[node] == [[{node: 0}], [{node: 1}]]:
            sources_perc += 1
    data["general"]["sources_perc"] = sources_perc

    # get number of minimal trap spaces
    min_trap = compute_trap_spaces(percolated_primes, type_="min")
    data["general"]["min_trap_count"] = len(min_trap)

    # get number of attractors
    stg = primes2stg(percolated_primes, update)
    scc_dag = get_scc_dag(stg)
    attractor_indices = get_attractor_states(scc_dag, as_indices=True, DEBUG=DEBUG)
    data["general"]["attractor_count"] = len(attractor_indices)


    ### grouping information

    T = get_transition_matrix(stg, DEBUG=DEBUG)
    
    # get sd grouping
    sd_indices = sd_grouping(percolated_bnet, DEBUG=DEBUG)
    data["general"]["sd_group_count"] = len([x for x in sd_indices if x])
    sd_index_lengths = [len(x) for x in sd_indices if x]

    data["general"]["sd_group_size_mean"] = np.mean(sd_index_lengths)
    data["general"]["sd_group_size_std"] = np.std(sd_index_lengths)

    Tsd = compress_matrix(T, sd_indices, DEBUG=DEBUG)
    sd_markov_chain = get_markov_chain(Tsd, sd_indices, DEBUG=DEBUG)
    data["general"]["sd_mc_edges_count"] = sd_markov_chain.number_of_edges()

    # get null grouping
    null_indices = null_grouping(percolated_bnet, DEBUG=DEBUG)
    data["general"]["null_group_count"] = len([x for x in null_indices if x])
    null_index_lengths = [len(x) for x in null_indices if x]

    data["general"]["null_group_size_mean"] = np.mean(null_index_lengths)
    data["general"]["null_group_size_std"] = np.std(null_index_lengths)

    Tnull = compress_matrix(T, null_indices, DEBUG=DEBUG)
    null_markov_chain = get_markov_chain(Tnull, null_indices, DEBUG=DEBUG)
    data["general"]["null_mc_edges_count"] = null_markov_chain.number_of_edges()

    # get random grouping
    random_group_counts = []
    random_group_size_means = []
    random_group_size_stds = []
    random_mc_edges_counts = []
    for i in range(num_runs):
        random_indices = random_grouping(sd_indices, null_indices, seed=i, DEBUG=DEBUG)
        random_group_counts.append(len([x for x in random_indices if x]))

        random_index_lengths = [len(x) for x in random_indices if x]
        random_group_size_means.append(np.mean(random_index_lengths))
        random_group_size_stds.append(np.std(random_index_lengths))

        Trandom = compress_matrix(T, random_indices, DEBUG=DEBUG)
        random_markov_chain = get_markov_chain(Trandom, random_indices, DEBUG=DEBUG)
        random_mc_edges_counts.append(random_markov_chain.number_of_edges())

    data["general"]["random_group_count_mean"] = np.mean(random_group_counts)
    data["general"]["random_group_size_mean"] = np.mean(random_group_size_means)
    data["general"]["random_group_size_stds_mean"] = np.mean(random_group_size_stds)

    data["general"]["random_mc_edges_count_mean"] = np.mean(random_mc_edges_counts)
    data["general"]["random_mc_edges_count_std"] = np.std(random_mc_edges_counts)

    return data

In [4]:
all_data = {}

# Iterate over the files in the directory
for filename in sorted(os.listdir(DIRECTORY)):

    # if filename != "n010_013.bnet":
    #     continue

    file_path = os.path.join(DIRECTORY, filename)
    
    # Check if it's a file (not a directory)
    if not os.path.isfile(file_path):
        continue

    # Open and read the file
    with open(file_path, 'r') as file:
        print(f"Reading file: {filename}")
        content = file.read()
        
    all_data[filename] = get_general_info(bnet=content, data={}, num_runs=NUM_RUNS, update=UPDATE, DEBUG=DEBUG)


Reading file: n010_000.bnet
Reading file: n010_001.bnet
Reading file: n010_002.bnet
Reading file: n010_003.bnet
Reading file: n010_004.bnet
duplicates={'000000001': [[{'n4': 0, 'n5': 0}, {'n0': 0, 'n5': 0}], [{'n2': 0, 'n6': 0, 'n8': 0}]], '000000101': [[{'n4': 0, 'n5': 0}, {'n0': 0, 'n5': 0}], [{'n1': 0, 'n2': 0, 'n6': 1}]], '000000111': [[{'n4': 0, 'n5': 0}, {'n0': 0, 'n5': 0}], [{'n1': 0, 'n2': 0, 'n6': 1}]], '000010001': [[{'n4': 0, 'n5': 0}, {'n0': 0, 'n5': 0}], [{'n2': 0, 'n6': 0, 'n8': 0}]], '000010100': [[{'n4': 0, 'n5': 0}, {'n0': 0, 'n5': 0}], [{'n1': 0, 'n2': 0, 'n6': 1}]], '000010101': [[{'n4': 0, 'n5': 0}, {'n0': 0, 'n5': 0}], [{'n1': 0, 'n2': 0, 'n6': 1}]], '000010110': [[{'n4': 0, 'n5': 0}, {'n0': 0, 'n5': 0}], [{'n1': 0, 'n2': 0, 'n6': 1}]], '000010111': [[{'n4': 0, 'n5': 0}, {'n0': 0, 'n5': 0}], [{'n1': 0, 'n2': 0, 'n6': 1}]], '000100001': [[{'n4': 0, 'n5': 0}, {'n0': 0, 'n5': 0}], [{'n2': 0, 'n6': 0, 'n8': 0}]], '000100100': [[{'n4': 0, 'n5': 0}, {'n0': 0, 'n5': 0}], 

In [5]:
# Define column headers (optional)
headers = [
    "model",
    "N", "sources",
    "N_p", "sources_p",
    "min_trap", "attractor",
    "sd_cnt", "null_cnt", 
    "sd_edge", "rand_edge",
    "sd_sz_std", "rand_sz_stds_mean"
]

# Compute width for alignment
col_width = 12

# Print header
print("".join(h.ljust(col_width) for h in headers))

for filename in all_data:
    g = all_data[filename]["general"]

    # Base values (always exist)
    row = [
        filename.split(".")[0],
        g["N"],
        g["sources"],
    ]

    # When N_perc = 0, print "-" for all remaining fields
    if g["N_perc"] == 0:
        row.extend(["-"] * (len(headers) - len(row)))
    else:
        # Helper: format floats to 2 decimals
        def fmt(x):
            if isinstance(x, float):
                return f"{x:.2f}"
            return x

        # Add remaining fields (formatted where needed)
        row.extend([
            g["N_perc"],
            g["sources_perc"],
            g["min_trap_count"],
            g["attractor_count"],

            g["sd_group_count"],
            g["null_group_count"],

            g["sd_mc_edges_count"],
            fmt(g["random_mc_edges_count_mean"]),

            fmt(g["sd_group_size_std"]),
            fmt(g["random_group_size_stds_mean"]),
        ])

    # Print the aligned row
    print("".join(str(item).ljust(col_width) for item in row))


model       N           sources     N_p         sources_p   min_trap    attractor   sd_cnt      null_cnt    sd_edge     rand_edge   sd_sz_std   rand_sz_stds_mean
n010_000    10          0           10          0           2           2           16          3           46          225.98      32.00       14.03       
n010_001    10          0           6           1           5           5           12          6           32          68.03       8.07        4.02        
n010_002    10          0           7           0           1           1           5           2           12          20.42       19.04       13.08       
n010_003    10          0           10          0           1           1           3           2           6           7.00        319.02      240.97      
n010_004    10          0           9           0           4           4           26          5           103         504.32      47.42       9.07        
n010_005    10          0           6           0    

In [ ]:
all_keys = ["filename"]
for filename in all_data:
    all_keys += list(all_data[filename]["general"].keys())
    break

rows = []
for filename in all_data:
    g = all_data[filename]["general"]

    row = {}
    row["filename"] = filename

    # Fill detected keys
    for key in all_keys:
        if key == "filename":
            continue

        if key in g:
            row[key] = g[key]  # raw value, unrounded
        else:
            row[key] = "-"     # missing field

    rows.append(row)

with open(OUTPUT_FILE, "w", newline="", encoding="utf-8") as f:
    writer = csv.DictWriter(f, fieldnames=all_keys)
    writer.writeheader()
    writer.writerows(rows)

print(f"Raw CSV saved to: {OUTPUT_FILE}")


Raw CSV saved to: basic_analysis.csv
